In [2]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [3]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [4]:
cube = Lar.cuboid([1,1,1])

([0.0 0.0 … 1.0 1.0; 0.0 0.0 … 1.0 1.0; 0.0 1.0 … 0.0 1.0], [[1, 2, 3, 4, 5, 6, 7, 8]])

## Vecchia funzione

In [5]:
function apply(affineMatrix, larmodel)
	data = collect(larmodel)
	V = data[1]

	m,n = size(V)
	W = [V; fill(1.0, (1,n))]
	V = (affineMatrix * W)[1:m,1:n]

	data[1] = V
	larmodel = Tuple(data)
	return larmodel
end

apply (generic function with 2 methods)

In [6]:
@btime apply(Lar.t(-0.5,-0.5),square)

  1.610 μs (14 allocations: 1.16 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [7]:
@btime apply(Lar.t(-0.5,-0.5,-0.5),cube)

  1.770 μs (14 allocations: 1.67 KiB)


([-0.5 -0.5 … 0.5 0.5; -0.5 -0.5 … 0.5 0.5; -0.5 0.5 … -0.5 0.5], [[1, 2, 3, 4, 5, 6, 7, 8]])

In [8]:
@benchmark apply(Lar.t(-0.5,-0.5,-0.5),cube)

BenchmarkTools.Trial: 
  memory estimate:  1.67 KiB
  allocs estimate:  14
  --------------
  minimum time:     1.760 μs (0.00% GC)
  median time:      1.870 μs (0.00% GC)
  mean time:        2.442 μs (3.61% GC)
  maximum time:     328.000 μs (97.11% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [9]:
@code_warntype apply(Lar.t(-0.5,-0.5,-0.5),cube) 

Variables
  #self#::Core.Compiler.Const(apply, false)
  affineMatrix::Array{Float64,2}
  larmodel@_3::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
  @_4::Any
  data::Array{Array,1}
  m::Any
  n::Any
  W::Any
  V::Any
  larmodel@_10::Tuple{Vararg{Array,N} where N}

Body::Tuple{Vararg{Array,N} where N}
1 ─       (larmodel@_10 = larmodel@_3)
│         (data = Main.collect(larmodel@_10::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}))
│         (V = Base.getindex(data, 1))
│   %4  = Main.size(V::Array)::Any
│   %5  = Base.indexed_iterate(%4, 1)::Any
│         (m = Core.getfield(%5, 1))
│         (@_4 = Core.getfield(%5, 2))
│   %8  = Base.indexed_iterate(%4, 2, @_4)::Any
│         (n = Core.getfield(%8, 1))
│   %10 = V::Array::Array
│   %11 = Core.tuple(1, n)::Core.Compiler.PartialStruct(Tuple{Int64,Any}, Any[Core.Compiler.Const(1, false), Any])
│   %12 = Main.fill(1.0, %11)::Array{Float64,2}
│         (W = Base.vcat(%10, %12))
│   %14 = (affineMatrix * W)::Any
│   %15 = (1:m)::Any
│  

## Modifiche

In [157]:
function applyv(affineMatrix, larmodel)
	data = collect(larmodel)
	V = data[1]

	m,n = size(V)
	W = [V; fill(1.0, (1,n))]
	V = (affineMatrix * W)[1:m,1:n]

	data[1] = V
	larmodel = Tuple(data)
	return larmodel
end

applyv (generic function with 2 methods)

In [158]:
@btime applyv(Lar.t(-0.5,-0.5,-0.5),cube)

  1.770 μs (14 allocations: 1.67 KiB)


([-0.5 -0.5 … 0.5 0.5; -0.5 -0.5 … 0.5 0.5; -0.5 0.5 … -0.5 0.5], [[1, 2, 3, 4, 5, 6, 7, 8]])

In [146]:
@code_warntype applyv(Lar.t(-0.5,-0.5,-0.5),cube)

Variables
  #self#::Core.Compiler.Const(applyv, false)
  affineMatrix::Array{Float64,2}
  larmodel::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
  @_4::Int64
  data::Array{Array,1}
  m::Int64
  n::Int64
  W::Array{_A,2} where _A
  V::Array{T,2} where T
  larmodel2::Tuple{Vararg{Array,N} where N}

Body::Tuple{Vararg{Array,N} where N}
1 ─       (data = Main.collect(larmodel))
│   %2  = Base.getindex(data, 1)::Array
│   %3  = Base.convert(Main.Matrix, %2)::Any
│         (V = Core.typeassert(%3, Main.Matrix))
│   %5  = Main.size(V)::Tuple{Int64,Int64}
│   %6  = Base.indexed_iterate(%5, 1)::Core.Compiler.PartialStruct(Tuple{Int64,Int64}, Any[Int64, Core.Compiler.Const(2, false)])
│         (m = Core.getfield(%6, 1))
│         (@_4 = Core.getfield(%6, 2))
│   %9  = Base.indexed_iterate(%5, 2, @_4::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Int64,Int64}, Any[Int64, Core.Compiler.Const(3, false)])
│         (n = Core.getfield(%9, 1))
│   %11 = V::Array{T,2} where T
│ 